In [ ]:
pip install raidionicsmaps@git+https://github.com/raidionics/raidionics_maps.git@master#raidionicsmaps

In [ ]:
#Download the test data
import os
import requests
import zipfile

test_data_url = 'https://github.com/raidionics/Raidionics-models/releases/download/1.2.0/Samples-RaidionicsMaps-UnitTest1.zip'

test_dir = os.path.join(os.getcwd(), 'unit_tests_results_dir')
os.makedirs(test_dir, exist_ok=True)

archive_dl_dest = os.path.join(test_dir, 'unittest1_data.zip')
if not os.path.exists(archive_dl_dest):
    headers = {}
    response = requests.get(test_patient_url, headers=headers, stream=True)
    response.raise_for_status()
    if response.status_code == requests.codes.ok:
        with open(archive_dl_dest, "wb") as f:
            for chunk in response.iter_content(chunk_size=1048576):
                f.write(chunk)
    with zipfile.ZipFile(archive_dl_dest, 'r') as zip_ref:
        zip_ref.extractall(test_dir)

assert(os.path.exists(os.path.join(test_dir, 'Cohort_UnitTest1')))

In [ ]:
import configparser
import os
import warnings
import logging

# Ignore all warnings
warnings.filterwarnings("ignore")
logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

test_dir = os.path.join(os.getcwd(), 'unit_tests_results_dir')

test_config = configparser.ConfigParser()
test_config.add_section('Default')
test_config.set('Default', 'task', 'heatmap')
test_config.set('Default', 'input_folder', os.path.join(test_dir, 'Cohort_UnitTest1'))
test_config.set('Default', 'output_folder', os.path.join(test_dir, 'Cohort_UnitTest1_Output'))
test_config.add_section('Maps')
test_config.set('Maps', 'gt_files_suffix', 'label_tumor.nii.gz')
test_config.set('Maps', 'use_registered_data', 'true')
test_config.set('Maps', 'extra_parameters_filename', os.path.join(test_dir, 'Cohort_UnitTest1', 'patients_parameters.csv'))
test_config.set('Maps', 'distribution_dense_parameters', 'Age,40')
test_config.set('Maps', 'distribution_categorical_parameters', 'Gender,')
test_config_filename = os.path.join(test_dir, 'unit_test_config.ini')
with open(test_config_filename, 'w') as outfile:
    test_config.write(outfile)

print("Running the heatmap generation unit test.\n")
from raidionicsmaps.compute import compute
compute(test_config_filename)

In [ ]:
# Inspecting the results
import os
import nibabel as nib
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

logging.getLogger().setLevel(logging.WARNING)

# Collecting the generated heatmap
heatmap_perc_filename = os.path.join(test_dir, 'Cohort_UnitTest1_Output', 'Heatmaps', 'heatmap_percentages.nii.gz')

# Loading the generated heatmap
heatmap_nib = nib.load(heatmap_perc_filename)
heatmap = heatmap_nib.get_fdata()[:]


# Plotting the heatmap for a single 2D slice
colors = ['black','green','blue','red']
cmap = matplotlib.colors.ListedColormap(colors)

fig, ax = plt.subplots(1, 1, figsize=(20, 15))
ax[0, 0].imshow(heatmap[:, :, 75], cmap="warm")

for i, title in enumerate(["Heatmap"]):
  ax[int(i/1), i%1].set_title(title)
plt.show()